# Named Entity Recognition Pipeline

El pipeline toma una URL de un feed en formato RSS, obtiene el título y descripción de los artículos en el feed, detecta las NER con un modelo pre-entrenado, y las muestra ordenadas por frecuencia de aparición.

### Versiones
Probado con:
* Almond 0.6.0
* Ammonite 1.6.7
* Scala library version **2.11.12** -- Copyright 2002-2017, LAMP/EPFL
* Java 1.8.0_282

Para ver más información ir a (Help -> About Scala Kernel)

## 1. Obtener texto

### 1.1 Importar librerías

In [2]:
// Equivalent of adding dependencies to maven or sbt files
// For example, to add "org.scalaj" %% "scalaj-http" % "2.4.2" 
import $ivy.`org.scalaj::scalaj-http:2.4.2`
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"
import $ivy.`org.scala-lang.modules::scala-xml:1.3.0`

import $ivy.$                              
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"

import $ivy.$                                        

In [3]:
import scalaj.http.{Http, HttpResponse}
import scala.xml.XML

import scalaj.http.{Http, HttpResponse}

import scala.xml.XML

### 1.1 Obtener el texto del RSS Feed

Realizamos una consulta HTTP, que nos devuelve una instancia de HTTPResponse. Dentro del atributo `body` de la HTTPResponse, se encuentra el texto del feed en formato XML. Luego, se parsea el XML para extraer los campos `title` y `description`.

In [4]:
// Tutorial https://alvinalexander.com/source-code/scala-how-to-http-download-xml-rss-feed-timeout/
// get the xml content using scalaj-http
val url = "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"
val response: HttpResponse[String] = Http(url)
  .timeout(connTimeoutMs = 2000, readTimeoutMs = 5000)
  .asString
val xmlString = response.body
// convert the `String` to a `scala.xml.Elem`
val xml = XML.loadString(xmlString)
// Extract text from title and description
val rssText = (xml \\ "item").map { item =>
    ((item \ "title").text ++ " " ++ (item \ "description").text)
}

url: String = "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"
response: HttpResponse[String] = HttpResponse(
  """<?xml version="1.0" encoding="utf-8"?>
<rss version="2.0" xmlns:atom="http://www.w3.org/2005/Atom" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:media="http://search.yahoo.com/mrss/" xmlns:sy="http://purl.org/rss/1.0/modules/syndication/"><channel><title>Chicago Tribune</title><link>https://www.chicagotribune.com</link><language>en-US</language><copyright>© 2021 Chicago Tribune</copyright><atom:link href="https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:%5Bnow-2d+TO+now%5D&amp;sort=display_date:desc" rel="self" type="application/rss+xml"/><description>Chicago Tribune News Feed</description><lastBuildDate>Sat, 22 May 2021 22:46:03 +0000</lastBuildDate><ttl>1</ttl><sy:updatePeriod>hourly</sy:updatePeriod><sy:updateFrequ

## 2. Detectar las entidades nombradas

### 2.1 Crear el modelo

El **modelo** es sólo la función `getNEs`, que recibe una lista de textos.
Para cada texto, se separa las palabras del texto usando los espacios, y considera que es una entidad nombrada si empieza con mayúscula.

Este código lista los signos de puntuación y algunas palabras comunes del inglés que se van a sacar del texto.

In [5]:
val STOPWORDS = Seq (
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you",
    "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she",
    "her", "hers", "herself", "it", "its", "itself", "they", "them", "your",
    "their", "theirs", "themselves", "what", "which", "who", "whom",
    "this", "that", "these", "those", "am", "is", "are", "was", "were",
    "be", "been", "being", "have", "has", "had", "having", "do", "does",
    "did", "doing", "a", "an", "the", "and", "but", "if", "or",
    "because", "as", "until", "while", "of", "at", "by", "for", "with",
    "about", "against", "between", "into", "through", "during", "before",
    "after", "above", "below", "to", "from", "up", "down", "in", "out",
    "off", "over", "under", "again", "further", "then", "once", "here",
    "there", "when", "where", "why", "how", "all", "any", "both", "each",
    "few", "more", "most", "other", "some", "such", "no", "nor", "not",
    "only", "own", "same", "so", "than", "too", "very", "s", "t", "can",
    "will", "just", "don", "should", "now", "on",
    // Contractions without '
    "im", "ive", "id", "Youre", "youd", "youve",
    "hes", "hed", "shes", "shed", "itd", "were", "wed", "weve",
    "theyre", "theyd", "theyve",
    "shouldnt", "couldnt", "musnt", "cant", "wont",
    // Common uppercase words
    "hi", "hello"
)
val punctuationSymbols = ".,()!?;:'`´\n"
val punctuationRegex = "\\" + punctuationSymbols.split("").mkString("|\\")

STOPWORDS: Seq[String] = List(
  "i",
  "me",
  "my",
  "myself",
  "we",
  "our",
  "ours",
  "ourselves",
  "you",
  "yours",
  "yourself",
  "yourselves",
  "he",
  "him",
  "his",
  "himself",
  "she",
  "her",
  "hers",
  "herself",
  "it",
  "its",
  "itself",
  "they",
  "them",
  "your",
  "their",
  "theirs",
  "themselves",
  "what",
  "which",
  "who",
  "whom",
  "this",
  "that",
  "these",
  "those",
  "am",
...
punctuationSymbols: String = """.,()!?;:'`´
"""
punctuationRegex: String = """\.|\,|\(|\)|\!|\?|\;|\:|\'|\`|\´|\
"""

In [6]:
class NERModel(STOPWORDS: Seq[String], punctuationSymbols: String, punctuationRegex: String) {
    // Extract Named Entitis
    def getNEsSingle(text: String): Seq[String] = {
      text.replaceAll(punctuationRegex, "").split(" ")
        .filter { word:String => word.length > 1 &&
                  Character.isUpperCase(word.charAt(0)) &&
                  !STOPWORDS.contains(word.toLowerCase) }.toSeq
    }
    def getNEs(textList: Seq[String]): Seq[Seq[String]] = {
        textList.map(getNEsSingle)
    }
    
    // Counts Named Entities
    def countNEs(result: Seq[Seq[String]]): Map[String, Int] = {
        result.flatten.foldLeft(Map.empty[String, Int]) {
         (count, word) => count + (word -> (count.getOrElse(word, 0) + 1)) }
    }
    
    // Sorts Named Entities
    def sortNEs(counts: Map[String, Int]): List[(String, Int)] = {
        counts.toList.sortBy(_._2)(Ordering[Int].reverse)
    } 
}

defined class NERModel

In [7]:
val model = new NERModel(STOPWORDS, punctuationSymbols, punctuationRegex)

model: NERModel = ammonite.$sess.cmd5$Helper$NERModel@27218727

### 2.2 Aplicar el "Modelo" a los datos

In [8]:
val result = model.getNEs(rssText)

result: Seq[Seq[String]] = List(
  ArrayBuffer(
    "Chicago",
    "White",
    "Sox",
    "New",
    "York",
    "Yankees",
    "Chicago",
    "White",
    "Sox",
    "Dylan",
    "Cease",
    "New",
    "York",
    "Yankees",
    "Saturday",
    "Yankee",
    "Stadium",
    "Sox"
  ),
  ArrayBuffer(
    "John",
    "Tavares",
    "Toronto",
    "Maple",
    "Leafs\u2019",
    "Montreal",
    "Canadiens",
    "Toronto",
    "John",
    "Tavares",
    "Montreal",
    "Canadiens",
    "Game"
  ),
  ArrayBuffer("NBA", "NBA", "Saturday"),
  ArrayBuffer(
    "Chicago",
...

## 3. Contar y ordenar las entidades

Concatenar todas las listas, contar cada Named Entity, y luego ordernar por frecuencia

In [23]:
val counts = model.countNEs(result)
val sortedNEs = model.sortNEs(counts)

counts: Map[String, Int] = Map(
  "Much" -> 1,
  "Gleyber" -> 1,
  "Mercedes" -> 4,
  "Parker" -> 2,
  "John" -> 2,
  "Yerm\u00edn" -> 4,
  "Foster\u2019s" -> 1,
  "Soccer" -> 1,
  "Canadiens" -> 2,
  "Column" -> 1,
  "Yankee" -> 2,
  "Sky" -> 4,
  "Nationals" -> 1,
  "Washington" -> 1,
  "Mariners" -> 2,
  "Rod\u00f3n" -> 1,
  "Skys" -> 1,
  "Confused" -> 1,
  "Louis" -> 3,
  "Cease" -> 1,
  "President" -> 1,
  "Foster" -> 1,
  "Olympic" -> 1,
  "Sox\u2019s" -> 2,
  "Rod\u00f3n\u2019s" -> 1,
  "Kyle" -> 1,
  "US" -> 2,
  "Baldelli" -> 2,
  "MVP" -> 1,
  "Minnesota" -> 2,
  "IOC" -> 1,
  "Matt" -> 2,
  "Maple" -> 1,
  "Red" -> 3,
  "Tavares" -> 2,
  "Tyler" -> 2,
  "What\u2019s" -> 1,
  "White" -> 13,
...
sortedNEs: List[(String, Int)] = List(
  ("Chicago", 29),
  ("White", 13),
  ("Sox", 13),
  ("Cubs", 8),
  ("Friday", 7),
  ("York", 5),
  ("New", 5),
  ("Mercedes", 4),
  ("Yerm\u00edn", 4),
  ("Sky", 4),
  ("Bears", 4),
  ("Yankees", 4),
  ("Louis", 3),
  ("Red", 3),
  ("Stadium", 3